# Peter Moss Acute Myeloid & Lymphoblastic Leukemia AI Research Project
## Acute Lymphoblastic Leukemia Arduino Nano 33 BLE Classifier

![Acute Lymphoblastic Leukemia Arduino Nano 33 BLE Classifier](../assets/images/project-banner.jpg)

# Introduction

The **Acute Lypmhoblastic Leukemia Arduino Nano 33 BLE Sense Classifier** is an experiment to explore how low powered microcontrollers, specifically the Arduino Nano 33 BLE Sense, can be used to detect Acute Lymphoblastic Leukemia. The Arduino Nano 33 BLE Sense is the latest Arduino Board  which supports  Tensorflow Lite, allowing machine learning on Arduino. 

The motivation for this project was to explore how low powered devices such as Arduino can be used to detect Acute Lymphoblastic Leukemia. The project will be submitted to the Tensorflow For Microcontroller Challenge and the Eyes on Edge: tinyML Vision Challenge.

# DISCLAIMER

_This project should be used for research purposes only. The purpose of the project is to show the potential of Artificial Intelligence for medical support systems such as diagnostic systems._

_Although the model is accurate and shows good results both on paper and in real world testing, it is trained on a small amount of data and needs to be trained on larger datasets to really evaluate it's accuracy._

_Developers that have contributed to this repository have experience in using Artificial Intelligence for detecting certain types of cancer. They are not doctors, medical or cancer experts._

# Software

To use this project you will need to install the following software:

## DEVELOPER TO LIST REQUIRED SOFTWARE

# Installation

Before you can begin this tutorial you need to follow the [installation guide](https://github.com/AMLResearchProject/ALL-Arduino-Nano-33-BLE-Sense-Classifier/blob/main/docs/installation/notebook-trainer.md) to install the relevant software and setup your environment.

# Import Modules

In [ ]:
# DEVELOPER TO IMPORT MODULES

# Set Global Variables

In [ ]:
# DEVELOPER TO SET GLOBAL VARIABLES

# Data

## DEVELOPER TO DESCRIBE DATASET USED

## Data Preprocessing 

In this part of the notebook we will create the methods used for processing the data ready for training.

### Testing Data

First we need to remove the test data that we will use for testing the classifier later from the training data folder. We remove these images before the training so that the classifier does not learn them during training. The following cell provides the function to move the test data to the **model/data/test** folder: 

In [ ]:
def remove_testing():
    """ Removes the testing images from the dataset. """

    for img in testData:
        original = trainDataDir+img
        destination = testDataDir+img
        pathlib.Path(original).rename(destination)
        print(original + " moved to " + destination)
        cv2.imwrite(destination, cv2.resize(cv2.imread(destination),
                                            (dim, dim)))
        print("Resized " + destination)

### Augmentation

We will create an augmented dataset based on the [Leukemia Blood Cell Image Classification Using Convolutional Neural Network](http://www.ijcte.org/vol10/1198-H0012.pdf "Leukemia Blood Cell Image Classification Using Convolutional Neural Network") by T. T. P. Thanh, Caleb Vununu, Sukhrob Atoev, Suk-Hwan Lee, and Ki-Ryong Kwon.

In [ ]:
## DEVELOPER TO COMPLETE AUGMENTATION FUNCTIONS

In [1]:
def preprocess():
    """ Preprocesses the data. """
    
    # DEVELOPER TO COMPLETE DATA PREPROCESSING CODE

## Data Preparation

In [ ]:
# DEVELOPER TO COMPLETE DATA PREPARATION CODE

## Data Split  

In [2]:
def get_split(data, labels):
    """ Splits the data and labels creating training and validation datasets. """

    # DEVELOPER TO COMPLETE DATA SPLIT CODE

## Process & Prepare Data

In [3]:
# DEVELOPER TO COMPLETE DATA PROCESS AND PREPARATION EXECUTION CODE

## Data Visualization  

In [4]:
# DEVELOPER TO COMPLETE DATA VISUALIZATION CODE

# Network
## Build Network  

In [5]:
def build_network():
    """ Builds the network. """

    # DEVELOPER TO COMPLETE NETWORK CODE

# Model
## Train Model 

In [ ]:
def train(X_train, y_train, X_test, y_test):
    """ Trains the model. """
    
    # DEVELOPER TO COMPLETE MODEL TRAINING CODE

## Save Model 

In [ ]:
def save_model_as_json(tf_model):
    """ Saves the model as JSON """

    # DEVELOPER TO COMPLETE SAVE MODEL CODE

## Save Weights

In [ ]:
def save_weights(tf_model):
    """ Saves the model weights """
    
    # DEVELOPER TO COMPLETE SAVE WEIGHTS CODE

## Evaluate Model  

In [ ]:
def evaluate(tf_model, X_test, y_test):
    """ Evaluates the model """

    # DEVELOPER TO COMPLETE EVALUATION CODE

### Predictions

In [ ]:
def predictions(tf_model, X_train, X_test):
    """ Gets predictions. """

    # DEVELOPER TO COMPLETE PREDICTIONS CODE

## Model Metrics 

### Accuracy

In [ ]:
def plot_accuracy(history):
    """ Plots the accuracy. """

    # DEVELOPER TO COMPLETE ACCURACY PLOT CODE

### Loss

In [ ]:
def plot_loss(history):
    """ Plots the loss. """

    # DEVELOPER TO COMPLETE LOSS PLOT CODE

### Precision

In [ ]:
def plot_precision(history):
    """ Plots the precision. """

    # DEVELOPER TO COMPLETE PRECISION PLOT CODE

### AUC

In [ ]:
def plot_auc(history):
    """ Plots the AUC. """

    # DEVELOPER TO COMPLETE AUC PLOT CODE

### Recall

In [ ]:
def plot_recall(history):
    """ Plots the recall. """

    # DEVELOPER TO COMPLETE RECALL PLOT CODE

## Confusion Matrix

In [6]:
def confusion_matrix_plot(y_test, test_preds):
    """ Plots the confusion matrix. """

    # DEVELOPER TO COMPLETE CONFUSION MATRIX PLOT CODE

# Figures Of Merit

In [7]:
def figures_of_merit(self):
    """ Calculates/prints the figures of merit.

    https://homes.di.unimi.it/scotti/all/
    """

    # DEVELOPER TO COMPLETE FIGURES OF MERIT CODE

# Testing

In [ ]:
def load():
    """ Loads the model """

    # DEVELOPER TO COMPLETE MDOEL LOAD CODE

In [ ]:
def reshape(img):
    """ Reshapes an image. """

    # DEVELOPER TO COMPLETE RESHAPE CODE

In [ ]:
def predict(model, img):
    """ Gets a prediction for an image. """

    # DEVELOPER TO COMPLETE PREDICTION CODE

In [ ]:
def test(model):
    """ Test mode

    Loops through the test directory and classifies the images.
    """

    # DEVELOPER TO COMPLETE TEST CODE

# Conclusion  

**DEVELOPER TO PROVIDE PROJECT CONCLUSION**


# Contributing
Asociación de Investigacion en Inteligencia Artificial Para la Leucemia Peter Moss encourages and welcomes code contributions, bug fixes and enhancements from the Github community.

Please read the [CONTRIBUTING](CONTRIBUTING.md "CONTRIBUTING") document for a full guide to forking our repositories and submitting your pull requests. You will also find our code of conduct in the [Code of Conduct](CODE-OF-CONDUCT.md) document.

## Contributors

    ## - DEVELOPER ADD CONTRIBUTORS

&nbsp;

# Versioning
We use [SemVer](https://semver.org/) for versioning.

&nbsp;

# License
This project is licensed under the **MIT License** - see the [LICENSE](/LICENSE "LICENSE") file for details.

&nbsp;

# Bugs/Issues
We use the [repo issues](issues "repo issues") to track bugs and general requests related to using this project. See [CONTRIBUTING](CONTRIBUTING.md "CONTRIBUTING") for more info on how to submit bugs, feature requests and proposals.